In [180]:
import pandas as pd

In [181]:
config = {"AGE": 70,
          "TEST_SIZE": 0.2,
          "max_depth": 4}

In [182]:
import json
file = open("config.json", "w")
json.dump(config, file)
file.close()

In [183]:
credits = pd.read_csv("data/train.csv")

In [184]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73363 entries, 0 to 73362
Data columns (total 12 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Id                                    73363 non-null  int64  
 1   target                                73363 non-null  int64  
 2   RevolvingUtilizationOfUnsecuredLines  73363 non-null  float64
 3   age                                   73363 non-null  object 
 4   NumberOfTime30-59DaysPastDueNotWorse  73363 non-null  int64  
 5   DebtRatio                             73363 non-null  float64
 6   MonthlyIncome                         58796 non-null  float64
 7   NumberOfOpenCreditLinesAndLoans       73363 non-null  int64  
 8   NumberOfTimes90DaysLate               73363 non-null  int64  
 9   NumberRealEstateLoansOrLines          73363 non-null  int64  
 10  NumberOfTime60-89DaysPastDueNotWorse  73363 non-null  int64  
 11  NumberOfDepende

In [185]:
credits.drop(columns=["Id"],inplace=True)

In [186]:
X = credits.drop(columns=["target"])
y = credits["target"]

In [187]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1453)
for train_index, test_index in split.split(X, y):
    strat_train_set = X.iloc[train_index]
    strat_train_y_set = y.iloc[train_index]
    strat_test_set = X.iloc[test_index]
    strat_test_y_set = y.iloc[test_index]

In [188]:
strat_train_set.columns = [x.lower().replace("-", "_") for x in strat_train_set.columns]
strat_train_set["age"] = strat_train_set["age"].str.replace("y", "").astype(int)

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_13252\4001000583.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strat_train_set["age"] = strat_train_set["age"].str.replace("y", "").astype(int)


In [189]:
strat_train_set.describe()

,revolvingutilizationofunsecuredlines,age,numberoftime30_59dayspastduenotworse,debtratio,monthlyincome,numberofopencreditlinesandloans,numberoftimes90dayslate,numberrealestateloansorlines,numberoftime60_89dayspastduenotworse,numberofdependents
count,58690.000000,58690.000000,58690.000000,58690.000000,4.709200e+04,58690.000000,58690.000000,58690.000000,58690.000000,57160.000000
mean,6.062712,52.351116,0.405878,361.378416,6.672907e+03,8.452803,0.249259,1.021230,0.224638,0.756788
std,249.987308,14.780156,4.010534,2363.109679,1.115598e+04,5.131586,3.984726,1.135982,3.970500,1.112765
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.030158,41.000000,0.000000,0.173936,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.156277,52.000000,0.000000,0.365473,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.557897,63.000000,0.000000,0.856136,8.300000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,29110.000000,105.000000,98.000000,329664.000000,1.560100e+06,57.000000,98.000000,26.000000,98.000000,13.000000


In [190]:
strat_train_set["revolvingutilizationofunsecuredlines"].quantile(0.997)

1.6600166111679548

In [191]:
strat_train_set[strat_train_set["revolvingutilizationofunsecuredlines"] <=strat_train_set["revolvingutilizationofunsecuredlines"].quantile(0.997)].shape

(58513, 10)

In [192]:
import numpy as np

In [193]:
#strat_train_set = strat_train_set[strat_train_set["revolvingutilizationofunsecuredlines"] <=strat_train_set["revolvingutilizationofunsecuredlines"].quantile(0.997)]

In [194]:
total_datos = len(strat_train_set)
for c in strat_train_set.columns:
    strat_train_set = strat_train_set[(strat_train_set[c] <=strat_train_set[c].quantile(0.997)) | (strat_train_set[c].isnull())]
    print("He borrado",c, total_datos-len(strat_train_set))
    total_datos = len(strat_train_set)

He borrado revolvingutilizationofunsecuredlines 177
He borrado age 122
He borrado numberoftime30_59dayspastduenotworse 135
He borrado debtratio 175
He borrado monthlyincome 139
He borrado numberofopencreditlinesandloans 166
He borrado numberoftimes90dayslate 118
He borrado numberrealestateloansorlines 145
He borrado numberoftime60_89dayspastduenotworse 167
He borrado numberofdependents 85


In [195]:
strat_train_set.shape[0]/58513

0.9786030454770734

In [196]:
strat_train_set["monthlyincome"].fillna(strat_train_set["monthlyincome"].median(), inplace=True)

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_13252\3905886359.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  strat_train_set["monthlyincome"].fillna(strat_train_set["monthlyincome"].median(), inplace=True)


In [197]:
strat_train_set["monthlyincome"].isnull().sum()

0

In [198]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1453)
for train_index, test_index in split.split(X, y):
    strat_train_set = X.iloc[train_index]
    strat_train_y_set = y.iloc[train_index]
    strat_test_set = X.iloc[test_index]
    strat_test_y_set = y.iloc[test_index]

In [199]:
strat_train_set.columns = [x.lower().replace("-", "_") for x in strat_train_set.columns]
strat_train_set["age"] = strat_train_set["age"].str.replace("y", "").astype(int)

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_13252\4001000583.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strat_train_set["age"] = strat_train_set["age"].str.replace("y", "").astype(int)


In [200]:
total_datos = len(strat_train_set)
for c in strat_train_set.columns:
    strat_train_set = strat_train_set[(strat_train_set[c] <=strat_train_set[c].quantile(0.997)) | (strat_train_set[c].isnull())]
    print("He borrado",c, total_datos-len(strat_train_set))
    total_datos = len(strat_train_set)

He borrado revolvingutilizationofunsecuredlines 177
He borrado age 122
He borrado numberoftime30_59dayspastduenotworse 135
He borrado debtratio 175
He borrado monthlyincome 139
He borrado numberofopencreditlinesandloans 166
He borrado numberoftimes90dayslate 118
He borrado numberrealestateloansorlines 145
He borrado numberoftime60_89dayspastduenotworse 167
He borrado numberofdependents 85


In [201]:
strat_train_y_set.shape

(58690,)

In [202]:
strat_train_y_set = strat_train_y_set[strat_train_set.index]

In [203]:
strat_train_y_set.shape

(57261,)

In [204]:
miss = strat_train_set.isnull().sum()

In [205]:
miss

revolvingutilizationofunsecuredlines        0
age                                         0
numberoftime30_59dayspastduenotworse        0
debtratio                                   0
monthlyincome                           11214
numberofopencreditlinesandloans             0
numberoftimes90dayslate                     0
numberrealestateloansorlines                0
numberoftime60_89dayspastduenotworse        0
numberofdependents                       1479
dtype: int64

In [206]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
imputer.fit(strat_train_set[["monthlyincome", "numberofdependents"]])

SimpleImputer(strategy='median')

In [207]:
imputer.statistics_

array([5400.,    0.])

In [208]:
imputer.feature_names_in_

array(['monthlyincome', 'numberofdependents'], dtype=object)

In [209]:
strat_train_set[imputer.feature_names_in_] = imputer.transform(strat_train_set[imputer.feature_names_in_])

In [210]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000, tol=0.01)
lr.fit(strat_train_set, strat_train_y_set)

LogisticRegression(max_iter=1000, tol=0.01)

In [211]:
from sklearn.model_selection import StratifiedShuffleSplit
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=1453)
for train_index, test_index in split.split(X, y):
    strat_train_set = X.iloc[train_index]
    strat_train_y_set = y.iloc[train_index]
    strat_test_set = X.iloc[test_index]
    strat_test_y_set = y.iloc[test_index]

In [212]:
strat_train_set.columns = [x.lower().replace("-", "_") for x in strat_train_set.columns]
strat_train_set["age"] = strat_train_set["age"].str.replace("y", "").astype(int)

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_13252\4001000583.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strat_train_set["age"] = strat_train_set["age"].str.replace("y", "").astype(int)


In [213]:
total_datos = len(strat_train_set)
for c in strat_train_set.columns:
    strat_train_set = strat_train_set[(strat_train_set[c] <=strat_train_set[c].quantile(0.997)) | (strat_train_set[c].isnull())]
    print("He borrado",c, total_datos-len(strat_train_set))
    total_datos = len(strat_train_set)

He borrado revolvingutilizationofunsecuredlines 177
He borrado age 122
He borrado numberoftime30_59dayspastduenotworse 135
He borrado debtratio 175
He borrado monthlyincome 139
He borrado numberofopencreditlinesandloans 166
He borrado numberoftimes90dayslate 118
He borrado numberrealestateloansorlines 145
He borrado numberoftime60_89dayspastduenotworse 167
He borrado numberofdependents 85


In [214]:
strat_train_y_set = strat_train_y_set[strat_train_set.index]

In [215]:
strat_train_set["age"].replace(70, np.nan, inplace=True)

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_13252\428098689.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  strat_train_set["age"].replace(70, np.nan, inplace=True)


In [216]:
miss = strat_train_set.isnull().sum()

In [217]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
imputer.fit(strat_train_set[miss[miss>0].index])

SimpleImputer(strategy='median')

In [218]:
strat_train_set[miss[miss>0].index] = imputer.transform(strat_train_set[miss[miss>0].index])

In [219]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000, tol=0.01)
lr.fit(strat_train_set, strat_train_y_set)

LogisticRegression(max_iter=1000, tol=0.01)

In [220]:
strat_train_set[imputer.feature_names_in_] = imputer.transform(strat_train_set[imputer.feature_names_in_])

In [221]:
strat_train_set.isnull().sum()

revolvingutilizationofunsecuredlines    0
age                                     0
numberoftime30_59dayspastduenotworse    0
debtratio                               0
monthlyincome                           0
numberofopencreditlinesandloans         0
numberoftimes90dayslate                 0
numberrealestateloansorlines            0
numberoftime60_89dayspastduenotworse    0
numberofdependents                      0
dtype: int64

In [222]:
strat_train_set["monthlyincome"].median()

5400.0

In [223]:
#strat_train_set["monthlyincome"] =imputer.transform(strat_train_set[["monthlyincome"]])

In [224]:
strat_test_set.columns = [x.lower().replace("-", "_") for x in strat_test_set.columns]
strat_test_set["age"] = strat_test_set["age"].str.replace("y", "").astype(int)

C:\Users\Jonatan\AppData\Local\Temp\ipykernel_13252\4102285739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  strat_test_set["age"] = strat_test_set["age"].str.replace("y", "").astype(int)


In [225]:
strat_test_set

,revolvingutilizationofunsecuredlines,age,numberoftime30_59dayspastduenotworse,debtratio,monthlyincome,numberofopencreditlinesandloans,numberoftimes90dayslate,numberrealestateloansorlines,numberoftime60_89dayspastduenotworse,numberofdependents
27701,0.262274,62,0,0.563700,7393.0,10,0,2,0,0.0
44082,0.033654,68,0,0.299618,3400.0,17,0,1,0,0.0
11263,0.395531,44,0,0.365998,3040.0,5,0,0,0,0.0
57940,0.819747,54,1,990.000000,NaN,8,0,0,0,0.0
13441,0.028397,65,0,1014.000000,NaN,7,0,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...
17098,0.004860,64,0,2.765293,800.0,11,0,2,0,0.0
32429,0.012469,67,0,0.197951,4000.0,6,0,1,0,0.0
12583,0.351248,54,0,2543.000000,NaN,18,0,2,0,2.0
42665,0.922954,43,1,0.288371,4677.0,9,0,0,0,0.0


In [226]:
#strat_test_set["monthlyincome"] = imputer.transform(strat_test_set[["monthlyincome"]])

In [227]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
imputer.fit(strat_train_set[["monthlyincome", "numberofdependents"]])

SimpleImputer(strategy='median')

In [228]:
datos = pd.read_csv("data/train.csv")

In [229]:
datos

,Id,target,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,44164,0,0.768219,68y,1,0.687930,2178.0,12,0,1,0,0.0
1,95356,0,0.289873,52y,0,0.385497,3750.0,16,0,1,0,1.0
2,108917,0,0.000000,54y,0,0.346503,6833.0,8,0,1,0,0.0
3,99039,0,0.046064,66y,1,0.407614,5200.0,6,0,1,0,1.0
4,116168,0,0.168771,45y,0,4702.000000,NaN,10,0,3,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
73358,25055,0,0.010468,45y,0,0.211663,12500.0,15,0,1,0,3.0
73359,105554,0,0.131250,42y,0,0.311538,5000.0,9,0,0,0,0.0
73360,129050,0,0.550527,59y,0,2777.000000,NaN,12,0,1,0,0.0
73361,94633,0,0.678651,31y,0,0.436990,3800.0,12,0,1,0,0.0


In [230]:
assert datos["target"].isnull().sum() == 0 

In [231]:
#assert datos["MonthlyIncome"].isnull().sum() == 0 

In [232]:
# Transformaciones de los datos, se hacen en el train y se aplican al train/test
# No hardcodear
# Utilizar assert

In [233]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [234]:
tree_1 = DecisionTreeClassifier(max_depth=2, min_samples_leaf=10, random_state=434)
tree_1.fit(strat_train_set, strat_train_y_set)
print(accuracy_score(strat_train_y_set, tree_1.predict(strat_train_set)))

0.9395225371544332


In [235]:
tree_2 = DecisionTreeClassifier(max_depth=3, min_samples_leaf=10, random_state=434)
tree_2.fit(strat_train_set, strat_train_y_set)
print(accuracy_score(strat_train_y_set, tree_2.predict(strat_train_set)))

0.9400115261696442


In [236]:
dict_modelos = {}
for depth in range(1,11):
    tree_tmp = DecisionTreeClassifier(max_depth=depth, min_samples_leaf=10, random_state=434)
    tree_tmp.fit(strat_train_set, strat_train_y_set)
    dict_modelos[depth] = tree_tmp

In [237]:
def stats(m, scores):
    for s in list(scores.items()):
        print(s[0],s[1](strat_train_y_set, m.predict(strat_train_set)))
    

In [238]:
def mi_metrica(y_true, y_pred):
    return (y_true*y_pred).sum()

In [239]:
metrics = { "accuracy": accuracy_score,
            "recall": recall_score,
            "mi_metrica": mi_metrica
}

In [240]:
[stats(tmp, metrics) for tmp in dict_modelos.values()]

accuracy 0.9387017341646147
recall 0.0
mi_metrica 0
accuracy 0.9395225371544332
recall 0.13133903133903133
mi_metrica 461
accuracy 0.9400115261696442
recall 0.04757834757834758
mi_metrica 167
accuracy 0.940797401372662
recall 0.12962962962962962
mi_metrica 455
accuracy 0.9416705960426818
recall 0.16638176638176638
mi_metrica 584
accuracy 0.9423342239918967
recall 0.15384615384615385
mi_metrica 540
accuracy 0.9433820575959204
recall 0.14301994301994303
mi_metrica 502
accuracy 0.9446743857075497
recall 0.20997150997150998
mi_metrica 737
accuracy 0.9457396832049737
recall 0.19572649572649573
mi_metrica 687
accuracy 0.9469272279562005
recall 0.24017094017094018
mi_metrica 843


[None, None, None, None, None, None, None, None, None, None]

In [241]:
from sklearn.model_selection import GridSearchCV
tree_rd = DecisionTreeClassifier(random_state=42)
param_grid = {
    'max_depth': range(2,13),
    'min_samples_leaf': range(1,31)
}

grid_search = GridSearchCV(tree_rd, # El modelo que quiero entrenar
                                   param_grid, # El grid de parámetros
                                   cv=5, # Folds del cross validation
                                   scoring="accuracy", # Métrica que quiero optimizar
                                   n_jobs = -1 # Paralelizar
                                  )

In [242]:
grid_search.fit(strat_train_set, strat_train_y_set)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(2, 13),
                         'min_samples_leaf': range(1, 31)},
             scoring='accuracy')

In [243]:
type(grid_search.best_estimator_)

sklearn.tree._classes.DecisionTreeClassifier

In [245]:
grid_search.best_params_

{'max_depth': 5, 'min_samples_leaf': 1}

In [246]:
grid_search.feature_names_in_

array(['revolvingutilizationofunsecuredlines', 'age',
       'numberoftime30_59dayspastduenotworse', 'debtratio',
       'monthlyincome', 'numberofopencreditlinesandloans',
       'numberoftimes90dayslate', 'numberrealestateloansorlines',
       'numberoftime60_89dayspastduenotworse', 'numberofdependents'],
      dtype=object)

In [ ]:
grid_search.classes_

array([0, 1], dtype=int64)

In [ ]:
grid_search.

In [ ]:
arbol = grid_search.best_estimator_

In [ ]:
arbol.predict(strat_train_set)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)